## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
0,0,Lorengau,PG,-2.0226,147.2712,80.11,82,38,8.01,scattered clouds
1,1,Namibe,AO,-15.1961,12.1522,66.04,91,64,6.53,broken clouds
2,2,Cidreira,BR,-30.1811,-50.2056,58.37,52,0,13.56,clear sky
3,3,Znamenskoye,RU,57.1281,73.8282,40.68,94,60,3.31,broken clouds
4,4,Sovetskiy,RU,61.3614,63.5842,39.40,92,90,3.47,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest temperature you want for a vacation?"))
max_temp = float(input("What is the highest temperature you want for a vacation?"))

What is the lowest temperature you want for a vacation?70
What is the highest temperature you want for a vacation?85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
vacation_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
0,0,Lorengau,PG,-2.0226,147.2712,80.11,82,38,8.01,scattered clouds
12,12,Santa Rosa,PH,14.3122,121.1114,79.11,86,96,6.04,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,73.04,71,3,14.56,clear sky
17,17,Mul,IN,20.0667,79.6667,76.08,95,100,4.47,overcast clouds
19,19,Ribeira Grande,PT,38.5167,-28.7000,73.80,73,75,20.71,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.count()

City_ID               255
City                  255
Country               254
Lat                   255
Lng                   255
Max Temp              255
Humidity              255
Cloudiness            255
Wind Speed            255
Current Conditions    255
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_vacation_df = vacation_cities_df.dropna()
clean_vacation_df.count

<bound method DataFrame.count of      City_ID            City Country      Lat       Lng  Max Temp  Humidity  \
0          0        Lorengau      PG  -2.0226  147.2712     80.11        82   
12        12      Santa Rosa      PH  14.3122  121.1114     79.11        86   
14        14         Rikitea      PF -23.1203 -134.9692     73.04        71   
17        17             Mul      IN  20.0667   79.6667     76.08        95   
19        19  Ribeira Grande      PT  38.5167  -28.7000     73.80        73   
..       ...             ...     ...      ...       ...       ...       ...   
659      659          Acarau      BR  -2.8856  -40.1200     81.21        66   
660      660         Shimoda      JP  34.6667  138.9500     70.11        74   
661      661       Mporokoso      ZM  -9.3727   30.1250     71.37        41   
662      662     Bloomington      US  44.8408  -93.2983     74.71        52   
672      672       Quelimane      MZ -17.8786   36.8883     72.73        79   

     Cloudiness  W

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_vacation_df[["City", "Country", "Max Temp", "Current Conditions", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
0,Lorengau,PG,80.11,scattered clouds,-2.0226,147.2712,
12,Santa Rosa,PH,79.11,overcast clouds,14.3122,121.1114,
14,Rikitea,PF,73.04,clear sky,-23.1203,-134.9692,
17,Mul,IN,76.08,overcast clouds,20.0667,79.6667,
19,Ribeira Grande,PT,73.80,broken clouds,38.5167,-28.7000,
21,Jamestown,US,80.19,few clouds,42.0970,-79.2353,
22,Buala,SB,80.20,light rain,-8.1450,159.5921,
24,Atuona,PF,76.87,clear sky,-9.8000,-139.0333,
26,Honiara,SB,77.00,clear sky,-9.4333,159.9500,
30,Goderich,CA,83.91,overcast clouds,43.7501,-81.7165,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:  
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Conditions</dt><dd>{Current Conditions}</dd>
<dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))